In [ ]:
# This version saves a md file
%pip install "camel-ai[all]==0.2.16"

In [ ]:
%pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
import textwrap
import datetime
import logging
from pathlib import Path

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("news_analysis_workflow.log", mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger("NewsAnalysis")

# Import CAMEL components
from camel.agents import ChatAgent
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.tasks import Task
from camel.toolkits import FunctionTool, SearchToolkit
from camel.types import ModelPlatformType, ModelType
from camel.societies.workforce import Workforce

# Set up API keys (replace with your actual keys)
os.environ["OPENAI_API_KEY"] = "your_api_key_here"
os.environ["GOOGLE_API_KEY"] = "your_api_key_here"
os.environ["SEARCH_ENGINE_ID"] = "your_search_engine_id"

def analyze_news_topic(topic):
    """Analyze a news topic from multiple perspectives using CAMEL Workforce"""
    logger.info(f"Starting analysis of topic: {topic}")

    # Create a fact-gathering agent with search capabilities
    logger.info("Creating Fact Gatherer agent")
    search_toolkit = SearchToolkit()
    search_tools = [
        FunctionTool(search_toolkit.search_google),
        FunctionTool(search_toolkit.search_duckduckgo),
    ]

    fact_gatherer_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O,
    )

    fact_gatherer_msg = BaseMessage.make_assistant_message(
        role_name="Fact Gatherer",
        content=textwrap.dedent("""
            You are a fact-gathering researcher who excels at finding factual information about news topics.
            Your role is to search for and compile objective facts about the given news topic.
            Present all information with source attribution where possible.

            Format your response with:
            1. A brief overview of the topic
            2. 5-7 specific facts with sources
            3. A "Key Facts Summary" at the end
        """),
    )

    fact_gatherer = ChatAgent(
        system_message=fact_gatherer_msg,
        model=fact_gatherer_model,
        tools=search_tools,
    )

    # Create progressive perspective agent
    logger.info("Creating Progressive Analyst agent")
    progressive_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O,
    )

    progressive_msg = BaseMessage.make_assistant_message(
        role_name="Progressive Analyst",
        content=textwrap.dedent("""
            You are a political analyst with a progressive/left-leaning perspective.
            You value social justice, equality, government intervention for public good, environmental protection, and progressive social policies.
            Analyze the given topic through your political lens, highlighting what's important to those sharing your worldview.

            Format your response with:
            1. How progressives view this issue
            2. Key aspects important from your perspective
            3. A "Core Concerns" section summarizing key considerations
        """),
    )

    progressive_agent = ChatAgent(
        system_message=progressive_msg,
        model=progressive_model,
    )

    # Create conservative perspective agent
    logger.info("Creating Conservative Analyst agent")
    conservative_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O,
    )

    conservative_msg = BaseMessage.make_assistant_message(
        role_name="Conservative Analyst",
        content=textwrap.dedent("""
            You are a political analyst with a conservative/right-leaning perspective.
            You value tradition, fiscal responsibility, limited government, free markets, individual liberty, and social stability.
            Analyze the given topic through your political lens, highlighting what's important to those sharing your worldview.

            Format your response with:
            1. How conservatives view this issue
            2. Key aspects important from your perspective
            3. A "Core Concerns" section summarizing key considerations
        """),
    )

    conservative_agent = ChatAgent(
        system_message=conservative_msg,
        model=conservative_model,
    )

    # Create centrist perspective agent
    logger.info("Creating Centrist Analyst agent")
    centrist_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O,
    )

    centrist_msg = BaseMessage.make_assistant_message(
        role_name="Centrist Analyst",
        content=textwrap.dedent("""
            You are a political analyst with a moderate/centrist perspective.
            You value pragmatism, compromise, evidence-based policy, incremental change, and balance between stability and progress.
            Analyze the given topic through your political lens, highlighting what's important to those sharing your worldview.

            Format your response with:
            1. How centrists view this issue
            2. Key aspects important from your perspective
            3. A "Core Concerns" section summarizing key considerations
        """),
    )

    centrist_agent = ChatAgent(
        system_message=centrist_msg,
        model=centrist_model,
    )

    # Create historical context agent
    logger.info("Creating Historical Context Analyst agent")
    historical_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O,
    )

    historical_msg = BaseMessage.make_assistant_message(
        role_name="Historical Context Analyst",
        content=textwrap.dedent("""
            You are a historian specializing in providing historical context for current events.
            Connect current news events to historical patterns, precedents, or similar situations from the past.

            Format your response with:
            1. Relevant historical precedents or analogies
            2. Similarities and differences to the current situation
            3. A "Historical Insights" section with key historical lessons
        """),
    )

    historical_agent = ChatAgent(
        system_message=historical_msg,
        model=historical_model,
    )

    # Create media bias analyst agent
    logger.info("Creating Media Bias Analyst agent")
    media_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O,
    )

    media_msg = BaseMessage.make_assistant_message(
        role_name="Media Bias Analyst",
        content=textwrap.dedent("""
            You are a media literacy expert specializing in identifying media biases and framing.
            Analyze how different news outlets are covering a topic, identifying patterns in coverage.

            Format your response with:
            1. How the topic is likely being covered across the political spectrum
            2. Specific framing techniques being used
            3. A "Media Literacy Tips" section for readers
        """),
    )

    media_agent = ChatAgent(
        system_message=media_msg,
        model=media_model,
    )

    # Create synthesis agent
    logger.info("Creating Synthesis Specialist agent")
    synthesis_model = ModelFactory.create(
        model_platform=ModelPlatformType.OPENAI,
        model_type=ModelType.GPT_4O,
    )

    synthesis_msg = BaseMessage.make_assistant_message(
        role_name="Synthesis Specialist",
        content=textwrap.dedent("""
            You are a synthesis expert specializing in creating balanced, insightful overviews of complex topics.
            Integrate multiple perspectives on a news event into a cohesive analysis.

            Your synthesis MUST include:
            1. A comprehensive summary (2-3 paragraphs)
            2. Areas of factual consensus across perspectives
            3. Key points of disagreement between perspectives
            4. "Key Takeaways" section with 5-7 bullet points
            5. "Conclusion" section with final thoughts

            Be specific and substantive, avoiding vague generalizations.
        """),
    )

    synthesis_agent = ChatAgent(
        system_message=synthesis_msg,
        model=synthesis_model,
    )

    # Create the workforce with all agents
    logger.info("Creating News Analysis Workforce")
    workforce = Workforce('News Analysis Team')

    workforce.add_single_agent_worker(
        'Fact Gatherer, a researcher who searches for and compiles verifiable facts about news topics',
        worker=fact_gatherer,
    ).add_single_agent_worker(
        'Progressive Analyst, a political analyst with progressive/left-leaning perspective',
        worker=progressive_agent,
    ).add_single_agent_worker(
        'Conservative Analyst, a political analyst with conservative/right-leaning perspective',
        worker=conservative_agent,
    ).add_single_agent_worker(
        'Centrist Analyst, a political analyst with moderate/centrist perspective',
        worker=centrist_agent,
    ).add_single_agent_worker(
        'Historical Context Analyst, a historian who specializes in connecting current events to historical patterns',
        worker=historical_agent,
    ).add_single_agent_worker(
        'Media Bias Analyst, a media literacy expert who analyzes framing and bias in news coverage',
        worker=media_agent,
    ).add_single_agent_worker(
        'Synthesis Specialist, an expert in creating balanced overviews that integrate multiple perspectives',
        worker=synthesis_agent,
    )

    # Create the task for the workforce
    logger.info("Creating news analysis task")
    task_content = f"""
    Provide a comprehensive multi-perspective analysis of the following news topic: "{topic}".

    Follow this workflow:
    1. First, the Fact Gatherer will research and provide factual background on the topic.
    2. Then, the Progressive, Conservative, and Centrist Analysts will each analyze the topic from their perspective.
    3. The Historical Context Analyst will connect the topic to relevant historical patterns or precedents.
    4. The Media Bias Analyst will examine how the topic is being covered in different media outlets.
    5. Finally, the Synthesis Specialist will create an integrated overview that:
       - Summarizes the key factual points
       - Identifies areas of agreement and disagreement between perspectives
       - Highlights the most important insights from all analyses
       - Provides clear conclusions and key takeaways
    """

    task = Task(
        content=task_content,
        id="0",
    )

    # Process the task
    logger.info("Processing task with workforce")
    task_result = workforce.process_task(task)
    logger.info("Task processing complete")

    # Create markdown file
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"news_analysis_{timestamp}.md"
    logger.info(f"Creating markdown output file: {filename}")

    markdown_content = f"""# Multi-perspective Analysis: {topic}
*Generated on {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}*

## Table of Contents
- [Executive Summary](#executive-summary)
- [Factual Background](#factual-background)
- [Progressive Perspective](#progressive-perspective)
- [Conservative Perspective](#conservative-perspective)
- [Centrist Perspective](#centrist-perspective)
- [Historical Context](#historical-context)
- [Media Coverage Analysis](#media-coverage-analysis)

## Executive Summary
{task_result.result}
"""

    # Write to file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(markdown_content)

    logger.info(f"Analysis saved to {filename}")
    return {
        "summary": task_result.result,
        "full_analysis_file": filename
    }

if __name__ == "__main__":
    # Replace with your topic of interest
    topic = "Recent Supreme Court ruling on social media regulation"
    logger.info(f"Starting analysis of topic: {topic}")

    # Run the analysis
    analysis_result = analyze_news_topic(topic)

    # Print summary to console
    print("\n\nEXECUTIVE SUMMARY")
    print("=" * 80)
    print(analysis_result["summary"])
    print("\nFull analysis saved to:", analysis_result["full_analysis_file"])
    logger.info("Program execution complete")

Worker node 133254694068944 (Fact Gatherer, a researcher who searches for and compiles verifiable facts about news topics) get task 0.0: Fact Gatherer will research and compile factual background information on the recent Supreme Court ruling on social media regulation.
Reply from Worker node 133254694068944 (Fact Gatherer, a researcher who searches for and compiles verifiable facts about news topics):

### Overview
The recent Supreme Court ruling on social media regulation involves significant decisions regarding the interaction between government entities and social media platforms. This ruling has implications for how government officials can engage with social media and the extent to which they can influence content moderation.

### Specific Facts
1. **Federal Government Communication**: The Supreme Court allowed the federal government to continue communicating with social media platforms regarding content moderation. This decision is part of a broader dispute involving the Biden a

In [ ]:
if __name__ == "__main__":
    # Replace with your topic of interest
    topic = "Recent HCU issue related to Government of Telangana"
    logger.info(f"Starting analysis of topic: {topic}")

    # Run the analysis
    analysis_result = analyze_news_topic(topic)

    # Print summary to console
    print("\n\nEXECUTIVE SUMMARY")
    print("=" * 80)
    print(analysis_result["summary"])
    print("\nFull analysis saved to:", analysis_result["full_analysis_file"])
    logger.info("Program execution complete")

Worker node 133254669675536 (Fact Gatherer, a researcher who searches for and compiles verifiable facts about news topics) get task 0.0: Fact Gatherer: Research and compile factual background information on the recent HCU issue related to the Government of Telangana.
Reply from Worker node 133254669675536 (Fact Gatherer, a researcher who searches for and compiles verifiable facts about news topics):

### Overview
The recent issue involving the Government of Telangana and the University of Hyderabad (HCU) revolves around a land dispute concerning 400 acres of land near the university. This has led to legal challenges, environmental concerns, and student protests.

### Specific Facts
1. **Land Ownership Dispute**: The Telangana government has officially stated that 400 acres of land near the University of Hyderabad belong to the state and not the university. This clarification was issued amid ongoing disputes about the land's ownership. [Source: The Hans India](https://www.thehansindia.c

In [ ]:
if __name__ == "__main__":
    # Replace with your topic of interest
    topic = "Piyush Goel comments on startups in india"
    logger.info(f"Starting analysis of topic: {topic}")

    # Run the analysis
    analysis_result = analyze_news_topic(topic)

    # Print summary to console
    print("\n\nEXECUTIVE SUMMARY")
    print("=" * 80)
    print(analysis_result["summary"])
    print("\nFull analysis saved to:", analysis_result["full_analysis_file"])
    logger.info("Program execution complete")

Worker node 133254670322576 (Fact Gatherer, a researcher who searches for and compiles verifiable facts about news topics) get task 0.0: Research and compile factual background information on Piyush Goel's comments regarding startups in India.
Reply from Worker node 133254670322576 (Fact Gatherer, a researcher who searches for and compiles verifiable facts about news topics):

### Overview
Piyush Goyal, India's Minister of Commerce and Industry, has made several comments regarding the startup ecosystem in India that have sparked debate and reactions from various stakeholders. His remarks have focused on the nature of work Indian startups are engaged in and their global competitiveness.

### Specific Facts
1. **Criticism of Startup Focus**: Piyush Goyal criticized Indian startups for focusing too much on consumer-centric businesses like grocery delivery and ice cream making, urging them to shift towards deep-tech sectors. He made these comments during the second edition of the Startup M